# Explore Kubeflow Katib Hyperparamter Optimization API

In [2]:
import sys
print(sys.version)

3.9.21 (main, Dec 11 2024, 10:21:40) 
[Clang 14.0.6 ]


In [3]:
!pip install git+https://github.com/kubeflow/katib.git#subdirectory=sdk/python/v1beta1
!pip install git+https://github.com/kubeflow/training-operator.git#subdirectory=sdk/python

  Cloning https://github.com/kubeflow/katib.git to /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-jkz5vj8a
  Running command git clone --filter=blob:none --quiet https://github.com/kubeflow/katib.git /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-jkz5vj8a
  Resolved https://github.com/kubeflow/katib.git to commit 40e1e651f2f665cf8fb523b0f848644dc3036a78
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/kubeflow/training-operator.git to /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-1g8szvmv
  Running command git clone --filter=blob:none --quiet https://github.com/kubeflow/training-operator.git /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-1g8szvmv
  Resolved https://github.com/kubeflow/training-operator.git to commit ee1b691acdb2d605ffe77935aa45c57ac047ed15
  Preparing metadata (setup.py) ... done


In [1]:
!pip install transformers peft

In [51]:
# from kubeflow.katib.types.trainer_resources import TrainerResources
# 
# 
# resources_per_trial = TrainerResources(
#     num_workers=4,                    # Number of distributed workers
#     num_procs_per_worker=2,           # Processes per worker
#     resources_per_worker={            # Resource allocation per worker
#         "gpu": 2,                     # Number of GPUs
#         "cpu": 5,                     # Number of CPUs
#         "memory": "10G",              # Memory allocation
#     },
# )


In [6]:
import kubeflow.katib as katib
from kubeflow.katib import KatibClient

from transformers import AutoModelForSequenceClassification, TrainingArguments
from peft import LoraConfig

from kubeflow.storage_initializer.hugging_face import (
    HuggingFaceModelParams,
    HuggingFaceDatasetParams,
    HuggingFaceTrainerParams,
)

In [16]:
hf_model = HuggingFaceModelParams(
    model_uri = "hf://meta-llama/Llama-3.2-1B",
    transformer_type = AutoModelForSequenceClassification,
)

# Train the model on 1000 movie reviews from imdb
# https://huggingface.co/datasets/stanfordnlp/imdb
hf_dataset = HuggingFaceDatasetParams(
    repo_id = "imdb",
    split = "train[:1000]",
)

hf_tuning_parameters = HuggingFaceTrainerParams(
    training_parameters = TrainingArguments(
        output_dir = "results",
        save_strategy = "no",
        hub_strategy="all_checkpoints",
        learning_rate = 1e-05, #katib.search.double(min=1e-05, max=5e-05),
        num_train_epochs=3,
    )
)

In [17]:
cl = KatibClient(namespace="kubeflow")

In [19]:
# Fine-tuning for Binary Classification
exp_name = "testllm"
cl.tune(
    name = exp_name,
    model_provider_parameters = hf_model,
    dataset_provider_parameters = hf_dataset,
    trainer_parameters = hf_tuning_parameters,
    objective_metric_name = "train_loss",
    objective_type = "minimize",
    algorithm_name = "random",
    max_trial_count = 10,
    parallel_trial_count = 2,
    resources_per_trial={
        "gpu": "2",
        "cpu": "4",
        "memory": "10G",
    },
)

cl.wait_for_experiment_condition(name=exp_name)

# Get the best hyperparameters.
print(cl.get_optimal_hyperparameters(exp_name))

Thank you for using `tune` API for LLM hyperparameter optimization. This feature is in the alpha stage. Kubeflow community is looking for your feedback. Please share your experience via #kubeflow-katib Slack channel or the Kubeflow Katib GitHub.
PVC 'testllm' already exists in namespace kubeflow.


ValueError: <HUB_TOKEN> is not a valid HubStrategy, please select one of ['end', 'every_save', 'checkpoint', 'all_checkpoints']